## Импорт библиотек

In [1]:
import torch
import torchvision.transforms as transforms
from torchvision.models.detection import fasterrcnn_mobilenet_v3_large_fpn, FasterRCNN_MobileNet_V3_Large_FPN_Weights
import cv2

import time

import numpy as np
from IPython.display import clear_output

## Вспомогательные операции

In [4]:
!mkdir input
!mkdir output

Для детекции объектов будет использоваться модель обученная на датасете COCO, поэтому нам нужны имена классов из датасета.

In [5]:
# Список имен классов в датасете COCO
coco_names = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]


## Создание функций для предсказания и отрисовки

In [6]:
# Цвет для отрисовки границ объектов
COLOR = [255, 0, 0]  # Красный

# Создаем транформацию объекта в вектор
transform = transforms.Compose([
    transforms.ToTensor(),
])

def predict(image, model, device, detection_threshold):
    '''Предсказание модели.

    Функция получает на вход кадр, модель, устройство и порог обнаружения. 
    Модель получает на вход кадр. Выходом модели являются классы, обнаруженные 
    в кадре, уровень уверенности в данном классе и границы объектов, найденных в кадре.
    Затем оставляются только предсказания для класса "человек" соответствующие заданному
    порогу. Функция возвращает границы обектов, классы и уровень уверенности.

    Args:
        image (numpy.ndarray): Входной кадр в формате numpy array.
        model (torch.nn.Module): Предобученная модель для предсказаний.
        device (torch.device): Устройство (CPU или CUDA), на котором выполняется предсказание.
        detection_threshold (float): Порог уверенности для отбора предсказаний.

    Returns:
        tuple: Кортеж, содержащий три элемента:
            - boxes (numpy.ndarray): Границы обнаруженных объектов.
            - classes (list of str): Классы обнаруженных объектов.
            - scores (numpy.ndarray): Уровень уверенности для обнаруженных объектов.
    '''

    # Переводим кадр в вектор и переносим на устройство
    image = transform(image).to(device)
    image = image.unsqueeze(0)  # добавляем размерность батча
    outputs = model(image)  # делаем предсказания для кадра

    # Получаем все предсказания классов
    pred_classes = [coco_names[i] for i in outputs[0]['labels'].cpu().numpy()]

    # Получаем все показатели уверенности для классов
    pred_scores = outputs[0]['scores'].detach().cpu().numpy()

    # Получаем все границы объектов
    pred_bboxes = outputs[0]['boxes'].detach().cpu().numpy()

    # Оставляем только предсказания для класса с людьми
    person_indices = [i for i, label in enumerate(
        outputs[0]['labels'].cpu().numpy()) if label == 1]

    # Выыбираем границы объектов, уверенность и класс только для людей
    person_boxes = pred_bboxes[person_indices]
    person_scores = pred_scores[person_indices]
    person_classes = [pred_classes[i] for i in person_indices]

    # Оставляем только объекты, уверенность в которых больше заданного порога
    boxes = person_boxes[person_scores >= detection_threshold].astype(np.int32)
    scores = person_scores[person_scores >= detection_threshold]
    classes = [person_classes[i] for i in range(
        len(person_classes)) if person_scores[i] >= detection_threshold]

    return boxes, classes, scores

def draw_boxes(boxes, classes, scores, image):
    '''Функция для отрисовки предсказанных объектов в кадре.

    Функция получает на вход границы объектов, классы, уровень уверенности и кадр.
    Идет отрисовка прямоугольника соответствующего границам обнаруженных объектов,
    также подписывается класс и уровень уверенности в нем.

    На выходе функции получается кадр с отрисованными отъектами.

    Args:
        boxes (numpy.ndarray): Границы обнаруженных объектов.
        classes (list of str): Классы обнаруженных объектов.
        scores (numpy.ndarray): Уровень уверенности для обнаруженных объектов.
        image (numpy.ndarray): Входной кадр в формате numpy array.

    Returns:
        numpy.ndarray: Кадр с отрисованными объектами.
    '''
    # Преобразование RGB в BGR цветовое пространство для правильной работы
    image = cv2.cvtColor(np.asarray(image), cv2.COLOR_BGR2RGB)
    # Для каждой коробки
    for i, box in enumerate(boxes):
        # Отображение границы
        cv2.rectangle(
            image,
            (int(box[0]), int(box[1])),
            (int(box[2]), int(box[3])),
            COLOR, 1
        )
        # Надпись
        cv2.putText(image, f'{classes[i]}: {scores[i]:.2f}', (int(box[0]), int(box[1]-5)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, COLOR, 2,
                    lineType=cv2.LINE_AA)
    return image


## Детекция людей на видео

In [8]:
# Выбор устройства
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Загрузка  предобученной модели
model = fasterrcnn_mobilenet_v3_large_fpn(
    weights=FasterRCNN_MobileNet_V3_Large_FPN_Weights.DEFAULT)

# Перевод модели в режим предсказания и перенос на устройство
model = model.eval().to(device)

# Указать путь к видеофайлу здесь
video_path = 'input/crowd.mp4'

# Захват видео
cap = cv2.VideoCapture(video_path)

if (cap.isOpened() == False):
    print('Ошибка в чтении видео. Проверьте путь')

# Ширина и высота кадра
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Создание объекта для сохранения видео
out = cv2.VideoWriter('/content/outputs/crowd_processed.mp4',
                      cv2.VideoWriter_fourcc(*'mp4v'), 30,
                      (frame_width, frame_height))

frame_count = 0  # Подсчет общего количества кадров
total_fps = 0  # Подсчет итогового FPS

# Пока видео не закончилось
while (cap.isOpened()):
    # Захватываем каждый кадр из видео
    ret, frame = cap.read()
    if ret == True:
        # Записываем начальное время для подсчета скорости работы
        start_time = time.time()
        with torch.no_grad():
            # Получить предсказания для текущего кадра
            boxes, classes, scores = predict(frame, model, device, 0.7)

        # Отрисовка границ объектов и предсказаний
        image = draw_boxes(boxes, classes, scores, frame)

        # Записываем время окончания обработки кадра
        end_time = time.time()
        # Считаем время обработки кадра
        fps = 1 / (end_time - start_time)
        # Добавляем к итоговому FPS
        total_fps += fps
        # Обновляем счетчик кадров
        frame_count += 1
        clear_output(wait=True)
        print(f"Количество кадров: {frame_count}, FPS: {fps}")

        # Преобразование BGR в RGB цветовое пространство для правильного отображения
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Запись кадра в видео
        out.write(image)

    else:
        break

# Закрытие видеопотока
cap.release()
# Закрытие видеофайла для правильной записи кадров
out.release()
# Закрытие всех кадров и окон
cv2.destroyAllWindows()

# Подсчет финального FPS
avg_fps = total_fps / frame_count
print(f"Средний FPS: {avg_fps:.3f}")

Количество кадров: 705, FPS: 9.016615252324394
Средний FPS: 10.430


При запуске модели на Kaggle с GPU P100 средний FPS равен 16.446.
Это в 2 раза меньше чем обработка в реальном времени. Для улучшения резульата можно предпринять следующие шаги:
- Использование другой архитектуры нейронной сети, например YOLO;
- Использование больших вычислительных мощностей.

Выводы по качеству работы модели:
- Модель достаточно точно определяет людей на близком и среднем расстоянии;
- Люди, которые находятся вдали уже не определяются моделью;
- При проходе человека за препядствием он теряется.

Для того чтобы исправить эти недостатки можно дообучить модель на другом датасете, который будет специализирован на людях на улице. Например, использовать данную модель как backbone, а поверх нее обучить модель, которая будет детектировать только людей.